---
## Python Packages & Directories
---

In [11]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 20)

# from smh import Session

from spag.read_data import *
from spag.convert import *
from spag.utils import *
from spag.read_data import get_solar
import spag.coordinates as coord

# import alexmods.read_data as rd

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"
data_dir = '/Users/ayelland/Research/metal-poor-stars/spag/spag/data/'


In [12]:
## Show all columns and rows of the dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows

---
## Check for Duplicates using Coordinates
---
Here, we will compare twoo datasets to see if there are any stars in common. To do so, we will find common RA and DEC values of the stars.

### Load the Datasets

In [13]:
## Datasets for Comparison
df1 = load_placco2014c(remove_atari=False, remove_sass=False, remove_dups=False, use_jinabase_sass=False, io=None)
df2 = load_atari()

### Create the comparison Dataframes

In [14]:
## Create an empty DataFrame with the desired columns
coord_df1 = pd.DataFrame(columns=['ID', 'RA_hms', 'DEC_dms', 'RA_deg', 'DEC_deg'])

## Drop rows with NaN values in RA or DEC
df1a = df1.dropna(subset=['RA_hms', 'DEC_dms'])

## Create a list to store rows before converting to DataFrame
tmp_list = []

for index, row in df1a.iterrows():
    name = row['Simbad_Identifier'] if 'Simbad_Identifier' in row else row['Name']

    # Convert RA to degrees/hms
    if "RA_hms" in row.index:
        ra_hms = row['RA_hms'].replace(" ", ":")
        ra_deg = float(coord.ra_hms_to_deg(ra_hms))
    else:
        ra_deg = float(row['RA_deg'])
        ra_hms = coord.ra_deg_to_hms(ra_deg)

    # Convert DEC to degrees/dms
    if "DEC_dms" in row.index:
        dec_hms = row['DEC_dms'].replace(" ", ":")
        dec_deg = float(coord.dec_dms_to_deg(dec_hms))
    else:
        dec_deg = float(row['DEC_deg'])
        dec_hms = coord.dec_deg_to_dms(dec_deg)

    # Append row data to the list
    tmp_list.append({'ID': name, 'RA_hms': ra_hms, 'DEC_dms': dec_hms, 'RA_deg': ra_deg, 'DEC_deg': dec_deg})

# Convert list to DataFrame and concatenate with coord_df1
coord_df1 = pd.concat([coord_df1, pd.DataFrame(tmp_list)], ignore_index=True)
display(coord_df1.head(20))


,ID,RA_hms,DEC_dms,RA_deg,DEC_deg
0,BD+02 3375,17:39:45.5948,+02:24:59.606,264.939978,2.416557
1,BD+03 740,05:01:16.6226,+04:06:37.021,75.319261,4.110284
2,BD+06 648,04:13:13.1126,+06:36:01.768,63.304636,6.600491
3,BD+10 2495,12:59:19.9306,+09:14:35.634,194.833044,9.243232
4,BD+17 3248,17:28:14.4690,+17:30:35.847,262.060287,17.509957
5,BD+23 3130,17:32:41.6247,+23:44:11.625,263.173436,23.736563
6,BD+24 1676,07:30:41.2662,+24:05:10.248,112.671943,24.086180
7,BD+44 493,02:26:49.7390,+44:57:46.523,36.707246,44.962923
8,BD-01 2582,11:53:37.3231,-02:00:36.728,178.405513,-2.010202
9,BD-04 3208,12:07:15.0721,-05:44:01.612,181.812800,-5.733781


In [15]:
## Create an empty DataFrame with the desired columns
coord_df2 = pd.DataFrame(columns=['ID', 'RA_hms', 'DEC_dms', 'RA_deg', 'DEC_deg'])

## Drop rows with NaN values in RA or DEC
df2a = df2.dropna(subset=['RA_hms', 'DEC_dms'])

## Create a list to store rows before converting to DataFrame
tmp_list = []

for index, row in df2a.iterrows():
    name = row['Simbad_Identifier'] if 'Simbad_Identifier' in row else row['Name']

    # Convert RA to degrees/hms
    if "RA_hms" in row.index:
        ra_hms = row['RA_hms'].replace(" ", ":")
        ra_deg = float(coord.ra_hms_to_deg(ra_hms))
    else:
        ra_deg = float(row['RA_deg'])
        ra_hms = coord.ra_deg_to_hms(ra_deg)

    # Convert DEC to degrees/dms
    if "DEC_dms" in row.index:
        dec_hms = row['DEC_dms'].replace(" ", ":")
        dec_deg = float(coord.dec_dms_to_deg(dec_hms))
    else:
        dec_deg = float(row['DEC_deg'])
        dec_hms = coord.dec_deg_to_dms(dec_deg)

    # Append row data to the list
    tmp_list.append({'ID': name, 'RA_hms': ra_hms, 'DEC_dms': dec_hms, 'RA_deg': ra_deg, 'DEC_deg': dec_deg})

# Convert list to DataFrame and concatenate with coord_df1
coord_df2 = pd.concat([coord_df2, pd.DataFrame(tmp_list)], ignore_index=True)
display(coord_df2.head(20))


,ID,RA_hms,DEC_dms,RA_deg,DEC_deg
0,BD-09 4396,16:28:56.1354,-10:14:57.644,247.233897,-10.249346
1,TYC 9527-1073-1,18:28:43.5444,-84:41:34.807,277.181435,-84.693002
2,TYC 7602-1143-1,05:52:15.7868,-39:53:18.441,88.065778,-39.888456
3,HE 0023-4825,00:25:50.3198,-48:08:27.012,6.459666,-48.140837
4,UCAC2 30544966,14:10:15.8809,-03:43:55.307,212.566170,-3.732030
5,CD-54 503,02:21:55.5935,-54:10:14.472,35.481640,-54.170687
6,CD-26 16470,23:05:50.5487,-25:57:22.308,346.460620,-25.956197
7,HD 119516,13:43:26.7251,+15:34:31.069,205.861355,15.575297
8,TYC 353-223-1,15:54:27.2897,+00:21:36.870,238.613707,0.360242
9,HD 23798,03:46:45.7220,-30:51:13.333,56.690508,-30.853704


### Perform the comparison

In [16]:
## Compare the coordinates of the Jinabase and LVDB datasets
common_dw_stars = {"Name1": [], "Name2": []}

for idx1, ra1 in coord_df1['RA_deg'].items():
    for idx2, ra2 in coord_df2['RA_deg'].items():

        # Compare RA and DEC values
        if np.isclose(ra1, ra2, atol=0.01):
            dec1 = coord_df1.loc[idx1, 'DEC_deg']
            dec2 = coord_df2.loc[idx2, 'DEC_deg']
            if np.isclose(dec1, dec2, atol=0.01):

                # Save the IDs of the common stars
                name1 = coord_df1.loc[idx1, 'ID']
                name2 = coord_df2.loc[idx2, 'ID']

                common_dw_stars["Name1"].append(name1)
                common_dw_stars["Name2"].append(name2)

                # Print the common stars
                print("ID:  ", name1,  " || ", name2)
                print("RA:  ", ra1,  " || ", ra2)
                print("DEC: ", dec1, " || ", dec2)
                print()
            


ID:   BPS BS 16928-0053  ||  BPS BS 16928-0053
RA:   185.61676666666668  ||  185.61676666666668
DEC:  34.19213611111111  ||  34.19213611111111

ID:   BPS CS 22186-0023  ||  BPS CS 22186-0023
RA:   64.93972166666666  ||  64.93972166666666
DEC:  -36.859994444444446  ||  -36.859994444444446

ID:   BPS CS 22948-0104  ||  BPS CS 22948-0104
RA:   327.94066833333335  ||  327.94066833333335
DEC:  -37.87524305555556  ||  -37.87524305555556

ID:   BPS CS 22960-0064  ||  BPS CS 22960-0064
RA:   336.00059541666667  ||  336.00059541666667
DEC:  -42.587785000000004  ||  -42.587785000000004

ID:   BPS CS 29506-0007  ||  BPS CS 29506-0007
RA:   320.1193920833333  ||  320.1193920833333
DEC:  -20.773043055555554  ||  -20.773043055555554

ID:   BPS CS 30306-0132  ||  BPS CS 30306-0132
RA:   228.57882958333332  ||  228.57882958333332
DEC:  7.450744444444445  ||  7.450744444444445

ID:   BPS CS 31079-0028  ||  BPS CS 31079-0028
RA:   45.25286083333334  ||  45.25286083333334
DEC:  6.275502222222222  ||  6.2

In [18]:
print('df1: ', common_dw_stars["Name1"])
print('df2: ', common_dw_stars["Name2"])

## Display the common stars in df1 and df2
name_key = 'Simbad_Identifier' if 'Simbad_Identifier' in df1.columns else 'Name'
print("Using name key: ", name_key)
df1_common = df1[df1[name_key].isin(common_dw_stars["Name1"])]
df2_common = df2[df2[name_key].isin(common_dw_stars["Name2"])]

print("df1 common stars: ", len(df1_common))
print("df2 common stars: ", len(df2_common))
display(df1_common)
display(df2_common)

df1:  ['BPS BS 16928-0053', 'BPS CS 22186-0023', 'BPS CS 22948-0104', 'BPS CS 22960-0064', 'BPS CS 29506-0007', 'BPS CS 30306-0132', 'BPS CS 31079-0028', 'HD   2796', 'HD  23798', 'HD 119516', 'HE 0017-4346', 'HE 0023-4825', 'TYC 4928-1438-1', 'HE 1300+0157', 'TYC 4961-1053-1', 'HE 1413-1954', 'HE 1424-0241', 'HE 2259-3407', 'HE 2318-1621', 'UCAC3 215-112497', '2MASS J12450268-0738469']
df2:  ['BPS BS 16928-0053', 'BPS CS 22186-0023', 'BPS CS 22948-0104', 'BPS CS 22960-0064', 'BPS CS 29506-0007', 'BPS CS 30306-0132', 'BPS CS 31079-0028', 'HD 2796', 'HD 23798', 'HD 119516', 'HE 0017-4346', 'HE 0023-4825', 'TYC 4928-1438-1', 'HE 1300+0157', 'TYC 4961-1053-1', 'HE 1413-1954', 'HE 1424-0241', 'HE 2259-3407', 'HE 2318-1621', 'UCAC3 215-112497', '2MASS J12450268-0738469']
Using name key:  Simbad_Identifier
df1 common stars:  21
df2 common stars:  21


,Name,Simbad_Identifier,RA_hms,DEC_dms,RA_deg,DEC_deg,Teff,logg,logL,[Fe/H],ul[N/Fe],[N/Fe],[C/Fe],[C/Fe]c,[C/Fe]f,ul[Sr/Fe],[Sr/Fe],ul[Ba/Fe],[Ba/Fe],I/O,Reference,Ref,epsc_c,MP_key,Ncap_key,C_key,epsc,epsc_f,[C/H],[C/H]f,[Sr/H],ul[Sr/H],[Ba/H],ul[Ba/H],[Sr/Ba],ul[Sr/Ba]
30,BS 16928-053,BPS BS 16928-0053,12:22:28.0240,+34:11:31.690,185.6168,34.19,4679,1.16,2.82,-2.93,NaN,1.05,-0.24,0.69,0.45,NaN,-0.19,NaN,-0.79,1,Yong+2013a,YON13a,0.69,VMP,,,5.26,5.95,-3.17,-2.48,-3.12,NaN,-3.72,NaN,0.60,NaN
55,CS 22186-023,BPS CS 22186-0023,04:19:45.5332,-36:51:35.980,64.9397,-36.86,5066,2.19,1.92,-2.72,NaN,NaN,0.26,0.01,0.27,NaN,-0.05,NaN,-0.98,1,Barklem+2005b,BAR05b,0.01,VMP,,,5.97,5.98,-2.46,-2.45,-2.77,NaN,-3.70,NaN,0.93,NaN
98,CS 22948-104,BPS CS 22948-0104,21:51:45.7604,-37:52:30.875,327.9407,-37.88,5000,2.30,1.79,-2.76,NaN,NaN,0.38,0.01,0.39,NaN,NaN,NaN,NaN,1,Masseron+2012,MAS12,0.01,VMP,,,6.05,6.06,-2.38,-2.37,NaN,NaN,NaN,NaN,NaN,NaN
116,CS 22960-064,BPS CS 22960-0064,22:24:00.1429,-42:35:16.026,336.0006,-42.59,5060,2.20,1.91,-2.77,NaN,1.35,0.14,0.01,0.15,NaN,NaN,NaN,-0.20,1,Roederer+2014a,ROE14a,0.01,VMP,,,5.80,5.81,-2.63,-2.62,NaN,NaN,-2.97,NaN,NaN,NaN
141,CS 29506-007,BPS CS 29506-0007,21:20:28.6541,-20:46:22.955,320.1194,-20.77,6522,3.94,0.61,-2.73,0.93,NaN,0.49,0.00,0.49,NaN,0.18,NaN,0.09,1,Yong+2013a,YON13a,0.00,VMP,,,6.19,6.19,-2.24,-2.24,-2.55,NaN,-2.64,NaN,0.09,NaN
158,CS 30306-132,BPS CS 30306-0132,15:14:18.9191,+07:27:02.680,228.5788,7.45,5047,2.16,1.95,-2.56,NaN,NaN,0.34,0.01,0.35,NaN,0.18,NaN,0.22,1,Yong+2013a,YON13a,0.01,VMP,,,6.21,6.22,-2.22,-2.21,-2.38,NaN,-2.34,NaN,-0.04,NaN
187,CS 31078-018,BPS CS 31079-0028,03:01:00.6866,+06:16:31.808,45.2529,6.28,5100,2.27,1.86,-2.99,NaN,-0.38,0.37,0.01,0.38,NaN,0.26,NaN,0.38,1,Yong+2013a,YON13a,0.01,VMP,,,5.81,5.82,-2.62,-2.61,-2.73,NaN,-2.61,NaN,-0.12,NaN
193,HD 2796,HD 2796,00:31:16.9147,-16:47:40.790,7.8205,-16.79,4923,1.84,2.22,-2.31,NaN,0.85,-0.50,0.17,-0.33,NaN,0.33,NaN,-0.14,1,Yong+2013a,YON13a,0.17,VMP,,,5.62,5.79,-2.81,-2.64,-1.98,NaN,-2.45,NaN,0.47,NaN
200,HD 23798,HD 23798,03:46:45.7220,-30:51:13.333,56.6905,-30.85,4450,1.06,2.83,-2.26,NaN,NaN,-0.59,0.74,0.15,NaN,NaN,NaN,NaN,1,Simmerer+2004,SIM04,0.74,VMP,,,5.58,6.32,-2.85,-2.11,NaN,NaN,NaN,NaN,NaN,NaN
211,HD 119516,HD 119516,13:43:26.7251,+15:34:31.069,205.8614,15.58,5382,2.47,1.75,-2.11,NaN,NaN,-1.19,0.01,-1.18,NaN,NaN,NaN,NaN,1,Simmerer+2004,SIM04,0.01,VMP,,,5.13,5.14,-3.30,-3.29,NaN,NaN,NaN,NaN,NaN,NaN


,JINA_ID,Simbad_Identifier,Reference,RA_hms,DEC_dms,epsfe,[Fe/H],epsc,ulc,[C/H],ul[C/H],[C/Fe],ul[C/Fe],ecc,Z_max,Name,Ref,I/O,Ncap_key,C_key,MP_key,Loc,System,RA_deg,DEC_deg,NLTE,alpha_key,Type,Vel,Vel_bibcode,U_mag,B_mag,V_mag,R_mag,I_mag,J_mag,H_mag,K_mag,Teff,logg,Fe/H,Vmic,Added_by,epsli,epsbe,epsn,epso,epsf,epsna,epsmg,epsal,epssi,epsp,epss,epsk,epsca,epssc,epsti1,epsv,epscr,epsmn,epsco,epsni,epscu,epszn,epsga,epsge,epsas,epsse,epsrb,epssr,epsy,epszr,epsnb,epsmo,epsru,epsrh,epspd,epsag,epscd,epssn,epste,epsba,epsla,epsce,epspr,epsnd,epssm,epseu,epsgd,epstb,epsdy,epsho,epser,epstm,epsyb,epslu,epshf,epsw,epsos,epsir,epspt,epsau,epshg,epspb,epsbi,epsth,epsu,epsca2,epsti,epsv2,epscr2,epsmn2,epsfe2,ulli,ulbe,uln,ulo,ulf,ulna,ulmg,ulal,ulsi,ulp,uls,ulk,ulca,ulsc,ulti1,ulv,ulcr,ulmn,ulfe,ulco,ulni,ulcu,ulzn,ulga,ulge,ulas,ulse,ulrb,ulsr,uly,ulzr,ulnb,ulmo,ulru,ulrh,ulpd,ulag,ulcd,ulsn,ulte,ulba,ulla,ulce,ulpr,ulnd,ulsm,uleu,ulgd,ultb,uldy,ulho,uler,ultm,ulyb,ullu,ulhf,ulw,ulos,ulir,ulpt,ulau,ulhg,ulpb,ulbi,ulth,ulu,ulca2,ulti,ulv2,ulcr2,ulmn2,ulfe2,[Li/H],[Be/H],[N/H],[O/H],[F/H],[Na/H],[Mg/H],[Al/H],[Si/H],[P/H],[S/H],[K/H],[Ca/H],[Sc/H],[Ti1/H],[V/H],[Cr/H],[Mn/H],[Co/H],[Ni/H],[Cu/H],[Zn/H],[Ga/H],[Ge/H],[As/H],[Se/H],[Rb/H],[Sr/H],[Y/H],[Zr/H],[Nb/H],[Mo/H],[Ru/H],[Rh/H],[Pd/H],[Ag/H],[Cd/H],[Sn/H],[Te/H],[Ba/H],[La/H],[Ce/H],[Pr/H],[Nd/H],[Sm/H],[Eu/H],[Gd/H],[Tb/H],[Dy/H],[Ho/H],[Er/H],[Tm/H],[Yb/H],[Lu/H],[Hf/H],[W/H],[Os/H],[Ir/H],[Pt/H],[Au/H],[Hg/H],[Pb/H],[Bi/H],[Th/H],[U/H],[Ca2/H],[Ti/H],[V2/H],[Cr2/H],[Mn2/H],[Fe2/H],ul[Li/H],ul[Be/H],ul[N/H],ul[O/H],ul[F/H],ul[Na/H],ul[Mg/H],ul[Al/H],ul[Si/H],ul[P/H],ul[S/H],ul[K/H],ul[Ca/H],ul[Sc/H],ul[Ti1/H],ul[V/H],ul[Cr/H],ul[Mn/H],ul[Fe/H],ul[Co/H],ul[Ni/H],ul[Cu/H],ul[Zn/H],ul[Ga/H],ul[Ge/H],ul[As/H],ul[Se/H],ul[Rb/H],ul[Sr/H],ul[Y/H],ul[Zr/H],ul[Nb/H],ul[Mo/H],ul[Ru/H],ul[Rh/H],ul[Pd/H],ul[Ag/H],ul[Cd/H],ul[Sn/H],ul[Te/H],ul[Ba/H],ul[La/H],ul[Ce/H],ul[Pr/H],ul[Nd/H],ul[Sm/H],ul[Eu/H],ul[Gd/H],ul[Tb/H],ul[Dy/H],ul[Ho/H],ul[Er/H],ul[Tm/H],ul[Yb/H],ul[Lu/H],ul[Hf/H],ul[W/H],ul[Os/H],ul[Ir/H],ul[Pt/H],ul[Au/H],ul[Hg/H],ul[Pb/H],ul[Bi/H],ul[Th/H],ul[U/H],ul[Ca2/H],ul[Ti/H],ul[V2/H],ul[Cr2/H],ul[Mn2/H],ul[Fe2/H],[Li/Fe],[Be/Fe],[N/Fe],[O/Fe],[F/Fe],[Na/Fe],[Mg/Fe],[Al/Fe],[Si/Fe],[P/Fe],[S/Fe],[K/Fe],[Ca/Fe],[Sc/Fe],[Ti1/Fe],[V/Fe],[Cr/Fe],[Mn/Fe],[Co/Fe],[Ni/Fe],[Cu/Fe],[Zn/Fe],[Ga/Fe],[Ge/Fe],[As/Fe],[Se/Fe],[Rb/Fe],[Sr/Fe],[Y/Fe],[Zr/Fe],[Nb/Fe],[Mo/Fe],[Ru/Fe],[Rh/Fe],[Pd/Fe],[Ag/Fe],[Cd/Fe],[Sn/Fe],[Te/Fe],[Ba/Fe],[La/Fe],[Ce/Fe],[Pr/Fe],[Nd/Fe],[Sm/Fe],[Eu/Fe],[Gd/Fe],[Tb/Fe],[Dy/Fe],[Ho/Fe],[Er/Fe],[Tm/Fe],[Yb/Fe],[Lu/Fe],[Hf/Fe],[W/Fe],[Os/Fe],[Ir/Fe],[Pt/Fe],[Au/Fe],[Hg/Fe],[Pb/Fe],[Bi/Fe],[Th/Fe],[U/Fe],[Ca2/Fe],[Ti/Fe],[V2/Fe],[Cr2/Fe],[Mn2/Fe],[Fe2/Fe],ul[Li/Fe],ul[Be/Fe],ul[N/Fe],ul[O/Fe],ul[F/Fe],ul[Na/Fe],ul[Mg/Fe],ul[Al/Fe],ul[Si/Fe],ul[P/Fe],ul[S/Fe],ul[K/Fe],ul[Ca/Fe],ul[Sc/Fe],ul[Ti1/Fe],ul[V/Fe],ul[Cr/Fe],ul[Mn/Fe],ul[Co/Fe],ul[Ni/Fe],ul[Cu/Fe],ul[Zn/Fe],ul[Ga/Fe],ul[Ge/Fe],ul[As/Fe],ul[Se/Fe],ul[Rb/Fe],ul[Sr/Fe],ul[Y/Fe],ul[Zr/Fe],ul[Nb/Fe],ul[Mo/Fe],ul[Ru/Fe],ul[Rh/Fe],ul[Pd/Fe],ul[Ag/Fe],ul[Cd/Fe],ul[Sn/Fe],ul[Te/Fe],ul[Ba/Fe],ul[La/Fe],ul[Ce/Fe],ul[Pr/Fe],ul[Nd/Fe],ul[Sm/Fe],ul[Eu/Fe],ul[Gd/Fe],ul[Tb/Fe],ul[Dy/Fe],ul[Ho/Fe],ul[Er/Fe],ul[Tm/Fe],ul[Yb/Fe],ul[Lu/Fe],ul[Hf/Fe],ul[W/Fe],ul[Os/Fe],ul[Ir/Fe],ul[Pt/Fe],ul[Au/Fe],ul[Hg/Fe],ul[Pb/Fe],ul[Bi/Fe],ul[Th/Fe],ul[U/Fe],ul[Ca2/Fe],ul[Ti/Fe],ul[V2/Fe],ul[Cr2/Fe],ul[Mn2/Fe],ul[Fe2/Fe],ll[Li/Fe],ll[Be/Fe],ll[C/Fe],ll[N/Fe],ll[O/Fe],ll[F/Fe],ll[Na/Fe],ll[Mg/Fe],ll[Al/Fe],ll[Si/Fe],ll[P/Fe],ll[S/Fe],ll[K/Fe],ll[Ca/Fe],ll[Sc/Fe],ll[Ti1/Fe],ll[V/Fe],ll[Cr/Fe],ll[Mn/Fe],ll[Co/Fe],ll[Ni/Fe],ll[Cu/Fe],ll[Zn/Fe],ll[Ga/Fe],ll[Ge/Fe],ll[As/Fe],ll[Se/Fe],ll[Rb/Fe],ll[Sr/Fe],ll[Y/Fe],ll[Zr/Fe],ll[Nb/Fe],ll[Mo/Fe],ll[Ru/Fe],ll[Rh/Fe],ll[Pd/Fe],ll[Ag/Fe],ll[Cd/Fe],ll[Sn/Fe],ll[Te/Fe],ll[Ba/Fe],ll[La/Fe],ll[Ce/Fe],ll[Pr/Fe],ll[Nd/Fe],ll[Sm/Fe],ll[Eu/Fe],ll[Gd/Fe],ll[Tb/Fe],ll[Dy/Fe],ll[Ho/Fe],ll[Er/Fe],ll[Tm/Fe],ll[Yb/Fe],l